# Matrix low-rank decomposition

## Contents

- [1. Imports](#Imports)
- [2. Train models with fixed rank for CV](#Train-models-with-fixed-rank-for-CV )
- [3. Predict with learned models](#Predict-with-learned-models)
- [4. Rank selection: build models](#Rank-selection:-build-models)
- [5. Rank selection: inspect accuracy](#Rank-selection:-inspect-accuracy)

[Back to Chemfin](../Chemfin.ipynb)

### Imports

The first cell with code includes almost all necessary inputs.

Required packages: [numpy](http://www.numpy.org/), [scikit-learn](http://scikit-learn.org/), [dcor](https://pypi.python.org/pypi/dcor).

[Back to contents](#Contents)

In [1]:
%env OMP_NUM_THREADS=4
%env MKL_NUM_THREADS=4

env: OMP_NUM_THREADS=4
env: MKL_NUM_THREADS=4


In [2]:
import sys
sys.path.append('../src/')

from computational_utils import reshape

import numpy as np
import pandas as pd
import copy
import time
import scipy

from sklearn.metrics import accuracy_score

from matrix import estimateMzPolarityFactors
from matrix import MatrixClassifierLCMS

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

import dcor

### Train models with fixed rank for CV

It is assumed that CV indices has been already precomputed with [Initialize](./1_Initialize.ipynb).

[Back to contents](#Contents)

In [3]:
data_dirname = '../data/'
model_dirname = '../models/matrix_decomposition/'
model_filename_prefix = 'model_md_snn'

filename_dataset = 'dataset.npz'
#filename_cv = 'cv_indices.npz'
filename_cv = 'physical_cv_indices_nc.npz'

maxitnum = 1000

rank = 25

df = np.load(data_dirname+filename_cv)
test_indices, train_indices = df['test_indices'], df['train_indices']

df = np.load(data_dirname+filename_dataset)
X, y = df['data'], df['label']
X = np.reshape(X, [X.shape[0], -1], order='F')
Nfeatures = X.shape[1]

tms = []
for ind in xrange(len(train_indices)):
    
    clf = MatrixClassifierLCMS(Nfeatures, rank, maxitnum=maxitnum)
    tic = time.clock()
    clf.fit(X[train_indices[ind]], y[train_indices[ind]], verbose=0)
    toc = time.clock()
    tms.append(toc-tic)
    clf.saveParameters(model_dirname+'rank='+str(rank)+'_'+model_filename_prefix+'_'+str(ind))
    np.savez_compressed(
        model_dirname+'rank='+str(rank)+'_times_train_'+model_filename_prefix,
        tms=tms
    )
    print "ind=%d rank=%d time=%.2fs" % (
        ind, rank, tms[-1]
    )

ind=0 rank=25 time=476.41s
ind=1 rank=25 time=482.65s
ind=2 rank=25 time=480.43s
ind=3 rank=25 time=546.35s
ind=4 rank=25 time=621.15s
ind=5 rank=25 time=565.02s
ind=6 rank=25 time=589.30s
ind=7 rank=25 time=610.07s
ind=8 rank=25 time=450.80s
ind=9 rank=25 time=560.44s
ind=10 rank=25 time=534.95s
ind=11 rank=25 time=477.58s
ind=12 rank=25 time=499.60s
ind=13 rank=25 time=527.62s
ind=14 rank=25 time=571.92s
ind=15 rank=25 time=478.47s
ind=16 rank=25 time=486.72s
ind=17 rank=25 time=445.55s
ind=18 rank=25 time=568.95s
ind=19 rank=25 time=538.53s
ind=20 rank=25 time=533.78s
ind=21 rank=25 time=613.13s
ind=22 rank=25 time=473.57s
ind=23 rank=25 time=452.21s
ind=24 rank=25 time=431.73s


In [4]:
model_dirname = '../models/matrix_decomposition/'
model_filename = 'model_md_snn.npz'

df = np.load(
    model_dirname+'rank='+str(rank)+'_times_train_'+model_filename
)

tms = df['tms']
print "Median of training time: %.2f s" % (np.median(tms))

Median of training time: 527.62 s


### Predict with learned models
Principle angle used as measure of distance between two column-spaces.
Requires additional imports of sklearn metrics.

[Back to contents](#Contents)

In [5]:
data_dirname = '../data/'
model_dirname = '../models/matrix_decomposition/'
model_filename_base = 'model_md_snn_'
results_dirname = '../results/'

filename_dataset = 'dataset.npz'
filename_dataset2 = 'test2.npz'
#filename_cv = 'cv_indices.npz'
filename_cv = 'physical_cv_indices_nc.npz'

rank = 25

df = np.load(data_dirname+filename_cv)
test_indices, train_indices = df['test_indices'], df['train_indices']

df = np.load(data_dirname+filename_dataset)
X, y = df['data'], df['label']
X = np.reshape(X, [X.shape[0], -1], order='F')

Nfeatures = X.shape[1]

df = np.load(data_dirname+filename_dataset2)
X_test2, y_test2 = df['data'], df['label']
X_test2 = np.reshape(X_test2, [X_test2.shape[0], -1], order='F')
y_test2 = reshape(y_test2, [-1, 1])

tms = []
accuracies = []
f1s = []
confusion_matrices = []

predicted_pa_test = []
predicted_pa_test2 = []
for ind in xrange(len(train_indices)):
    clf = MatrixClassifierLCMS(Nfeatures, rank)
    model_filename = 'rank='+str(rank)+'_'+model_filename_base+str(ind)+'.npz'
    print model_filename
    clf.loadParameters(model_dirname+model_filename)
    
    tic = time.clock()
    y_train_pred = clf.predict(X[train_indices[ind]])
    toc = time.clock()
    tms_loc = [toc-tic]
    acc_loc = [accuracy_score(y[train_indices[ind]], y_train_pred)]
    f1_loc = [f1_score(y[train_indices[ind]], y_train_pred, average='weighted')]
    
    tic = time.clock()
    y_test_pred, y_test_pred_explicit = clf.predict(X[test_indices[ind]], return_all=True)
    toc = time.clock()
    tms_loc.append(toc-tic)
    conf_mat = confusion_matrix(y[test_indices[ind]], y_test_pred)
    acc_loc.append( accuracy_score(y[test_indices[ind]], y_test_pred) )
    f1_loc.append( f1_score(y[test_indices[ind]], y_test_pred, average='weighted') )
    
    y_test2_pred, y_test2_pred_explicit = clf.predict(X_test2, return_all=True)
    acc_loc.append( accuracy_score(y_test2, y_test2_pred) )
    f1_loc.append( f1_score(y_test2, y_test2_pred, average='weighted') )
    
    y_test_pred_explicit = y_test_pred_explicit.assign(TRUE=y[test_indices[ind]])
    predicted_pa_test.append( y_test_pred_explicit.values )
    y_test2_pred_explicit = y_test2_pred_explicit.assign(TRUE=y_test2)
    predicted_pa_test2.append( y_test2_pred_explicit.values )
    
    tms.append(tms_loc)
    confusion_matrices.append(conf_mat)
    accuracies.append(acc_loc)
    f1s.append(f1_loc)
    np.savez_compressed(
        results_dirname+'rank='+str(rank)+model_filename_base+'+CCA',
        tms=tms, confusion_matrices=confusion_matrices, accuracies=accuracies,
        f1s=f1s, classes=clf.classes, predicted_pa_test=predicted_pa_test,
        predicted_pa_test2=predicted_pa_test2
    )
    print "ind=%d rank=%d time=%.2fs/%.2fs acc=%.4f/%.4f/%.4f f1=%.4f/%.4f/%.4f" % (
        ind, rank, tms[-1][0], tms[-1][1], acc_loc[0], acc_loc[1], acc_loc[2],
        f1_loc[0], f1_loc[1], f1_loc[2]
    )

rank=25_model_md_snn_0.npz


/home/hariyuki/apd/lib/python2.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/hariyuki/apd/lib/python2.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


ind=0 rank=25 time=433.65s/39.17s acc=0.9947/0.6558/0.8409 f1=0.9948/0.6475/0.8760
rank=25_model_md_snn_1.npz
ind=1 rank=25 time=439.42s/33.25s acc=0.9909/0.7655/0.8182 f1=0.9911/0.7526/0.8516
rank=25_model_md_snn_2.npz
ind=2 rank=25 time=453.33s/35.40s acc=0.9894/0.7744/0.7500 f1=0.9895/0.7644/0.8035
rank=25_model_md_snn_3.npz
ind=3 rank=25 time=536.47s/30.03s acc=0.9931/0.8053/0.8182 f1=0.9932/0.7882/0.8413
rank=25_model_md_snn_4.npz
ind=4 rank=25 time=488.93s/26.65s acc=0.9911/0.7757/0.8182 f1=0.9913/0.7638/0.8413
rank=25_model_md_snn_5.npz
ind=5 rank=25 time=418.92s/36.17s acc=0.9895/0.6883/0.8182 f1=0.9897/0.6776/0.8516
rank=25_model_md_snn_6.npz
ind=6 rank=25 time=438.03s/34.58s acc=0.9937/0.7655/0.8409 f1=0.9939/0.7458/0.8787
rank=25_model_md_snn_7.npz
ind=7 rank=25 time=446.78s/32.12s acc=0.9962/0.7444/0.8182 f1=0.9962/0.7297/0.8413
rank=25_model_md_snn_8.npz
ind=8 rank=25 time=468.19s/37.27s acc=0.9915/0.8142/0.7955 f1=0.9917/0.8121/0.8471
rank=25_model_md_snn_9.npz
ind=9 rank

In [5]:
model_dirname = '../models/matrix_decomposition/'
model_filename_base = 'model_md_snn_'
results_dirname = '../results/'

rank = 25

df = np.load( results_dirname+'rank='+str(rank)+model_filename_base+'+CCA.npz' )


accuracies = np.median(df['accuracies'], axis=0)
f1s = np.median(df['f1s'], axis=0)
tms = df['tms']

numSampPerFold = map(lambda x: float(len(x)), test_indices)
tms[:, 1] /= numSampPerFold

tms = np.median(tms, axis=0)

test_accuracy = df['accuracies']

dataDict = {
    'train': [accuracies[0], f1s[0]],
    'test': [accuracies[1], f1s[1]],
    'test2': [accuracies[2], f1s[2]],
    'index': ['accuracy', 'F1']
}

print 'One sample (from test) prediction time: %.2f s' % (tms[1])

table = pd.DataFrame(dataDict)
table.set_index('index')
table


One sample (from test) prediction time: 0.29 s


,index,test,test2,train
0,accuracy,0.752212,0.818182,0.992746
1,F1,0.740961,0.851641,0.992820


One sample (from test) prediction time: 2.15 s


,index,test,test2,train
0,accuracy,0.754386,0.818182,0.992188
1,F1,0.744137,0.851641,0.992374


### Rank selection: build models

Grid search approach followed by inspection of accuracy (one times repeated K-fold)

[Back to contents](#Contents)

In [9]:
data_dirname = '../data/'
model_dirname = '../models/matrix_decomposition/rank_selection/'
model_filename_prefix = 'model_md_snn'

filename_dataset = 'dataset.npz'
#filename_cv = 'cv_indices.npz'
filename_cv = 'physical_cv_indices_nc.npz'

maxitnum = 1000
n_splits = 5

rank_max = 25
ranks = np.arange(rank_max) + 1

df = np.load(data_dirname+filename_cv)
test_indices, train_indices = df['test_indices'], df['train_indices']

df = np.load(data_dirname+filename_dataset)
X, y = df['data'], df['label']
X = np.reshape(X, [X.shape[0], -1], order='F')

Nfeatures = X.shape[1]

tms = []
for ind in xrange(n_splits):
    for i in xrange(len(ranks)):
        rank = ranks[i]
        clf = MatrixClassifierLCMS(Nfeatures, rank, maxitnum=maxitnum, Lambda=1e-3)
        tic = time.clock() 
        clf.fit(X[train_indices[ind]], y[train_indices[ind]], verbose=0)
        toc = time.clock()
        tms.append(toc-tic)
        clf.saveParameters(model_dirname+'rank='+str(rank)+'_'+model_filename_prefix+'_'+str(ind))
        np.savez_compressed(
            model_dirname+'rank='+str(rank)+'_times_train_'+model_filename_prefix,
            tms=tms
        )
        print "ind=%d rank=%d time=%.2fs" % (
            ind, rank, tms[-1]
        )

ind=0 rank=1 time=36.83s
ind=0 rank=2 time=7.32s
ind=0 rank=3 time=21.48s
ind=0 rank=4 time=41.20s
ind=0 rank=5 time=64.07s
ind=0 rank=6 time=86.91s
ind=0 rank=7 time=108.15s
ind=0 rank=8 time=126.07s
ind=0 rank=9 time=148.36s
ind=0 rank=10 time=170.90s
ind=0 rank=11 time=200.50s
ind=0 rank=12 time=218.00s
ind=0 rank=13 time=245.60s
ind=0 rank=14 time=275.69s
ind=0 rank=15 time=315.44s
ind=0 rank=16 time=303.46s
ind=0 rank=17 time=319.66s
ind=0 rank=18 time=354.12s
ind=0 rank=19 time=380.76s
ind=0 rank=20 time=414.56s
ind=0 rank=21 time=435.74s
ind=0 rank=22 time=449.02s
ind=0 rank=23 time=480.51s
ind=0 rank=24 time=495.39s
ind=0 rank=25 time=431.65s
ind=1 rank=1 time=44.08s
ind=1 rank=2 time=10.12s
ind=1 rank=3 time=21.91s
ind=1 rank=4 time=44.33s
ind=1 rank=5 time=63.30s
ind=1 rank=6 time=86.26s
ind=1 rank=7 time=105.77s
ind=1 rank=8 time=128.77s
ind=1 rank=9 time=153.71s
ind=1 rank=10 time=164.95s
ind=1 rank=11 time=186.46s
ind=1 rank=12 time=205.16s
ind=1 rank=13 time=229.07s
ind=1

### Rank selection: inspect accuracy

[Back to contents](#Contents)

In [ ]:
metrics = ['cca', 'dcor']

metric_name = metrics[0]

if metric_name == 'cca':
    metric = None
elif metric_name == 'dcor':
    metric = dcor.distance_correlation
else:
    raise NotImplementedError
    

data_dirname = '../data/'
model_dirname = '../models/matrix_decomposition/rank_selection/'
model_filename_base = 'model_md_snn_'
results_dirname = '../results/'
model_filename_prefix = 'model_md_snn_'


filename_dataset = 'dataset.npz'
filename_dataset2 = 'test2.npz'
#filename_cv = 'cv_indices.npz'
filename_cv = 'physical_cv_indices_nc.npz'

n_splits = 5
rank_max = 25
ranks = np.arange(rank_max) + 1
rank_pol = 2

df = np.load(data_dirname+filename_cv)
test_indices, train_indices = df['test_indices'], df['train_indices']

df = np.load(data_dirname+filename_dataset)
X, y = df['data'], df['label']
X = np.reshape(X, [X.shape[0], -1], order='F')

Nfeatures = X.shape[1]
tms = []
accuracies = []
f1s = []
for ind in xrange(n_splits):
    tms_l = []
    accuracies_l = []
    f1s_l = []
    for i in xrange(len(ranks)):
        rank = ranks[i]
        
        clf = MatrixClassifierLCMS(Nfeatures, rank)
        model_filename = 'rank='+str(rank)+'_'+model_filename_base+str(ind)+'.npz'
        print model_filename
        clf.loadParameters(model_dirname+model_filename)

        tic = time.clock()
        y_train_pred = clf.predict(X[train_indices[ind]], metric=metric)
        toc = time.clock()
        tms_loc = [toc-tic]
        acc_loc = [accuracy_score(y[train_indices[ind]], y_train_pred)]
        f1_loc = [f1_score(y[train_indices[ind]], y_train_pred, average='weighted')]

        tic = time.clock()
        y_test_pred = clf.predict(X[test_indices[ind]], metric=metric)
        toc = time.clock()
        tms_loc.append(toc-tic)
        acc_loc.append( accuracy_score(y[test_indices[ind]], y_test_pred) )
        f1_loc.append( f1_score(y[test_indices[ind]], y_test_pred, average='weighted') )


        tms_l.append(tms_loc)
        accuracies_l.append(acc_loc)
        f1s_l.append(f1_loc)
    accuracies.append(accuracies_l)
    f1s.append(f1s_l)
    tms.append(tms_l)
    fnm_tmp = 'rank_selection_metric='+metric_name+'_'
    fnm_tmp += model_filename_prefix+'+'+metric_name
    np.savez_compressed(
        results_dirname+fnm_tmp,
        tms=tms, accuracies=accuracies, f1s=f1s
    )
accuracies = np.array(accuracies)
f1s = np.array(f1s)
tms = np.array(tms)

print "Accuracies:"
print np.median(accuracies, axis=0)
print "F1-measures:"
print np.median(f1s, axis=0)
print "Prediction time (for all samples):"
print np.median(np.sum(tms, axis=-1), axis=0)

In [11]:
print "Accuracies:"
print np.median(accuracies, axis=0)
print "F1-measures:"
print np.median(f1s, axis=0)
print "Prediction time (for all samples):"
print np.median(np.sum(tms, axis=-1), axis=0)

Accuracies:
[[0.43557642 0.24675325]
 [0.63641686 0.40689655]
 [0.80229525 0.52336449]
 [0.87636933 0.61061947]
 [0.91159251 0.67256637]
 [0.93676815 0.69172932]
 [0.94496487 0.71428571]
 [0.95667447 0.72180451]
 [0.96327212 0.72932331]
 [0.96906667 0.73684211]
 [0.9728     0.72932331]
 [0.976      0.7443609 ]
 [0.97716628 0.73684211]
 [0.98174231 0.73684211]
 [0.9827856  0.7443609 ]
 [0.98360656 0.73684211]
 [0.98629355 0.7518797 ]
 [0.98711944 0.75862069]
 [0.98773333 0.7518797 ]
 [0.98748044 0.7518797 ]
 [0.98914906 0.77241379]
 [0.99113198 0.76551724]
 [0.99029126 0.76691729]
 [0.99113198 0.76551724]
 [0.99113198 0.77443609]]
F1-measures:
[[0.48456933 0.25522919]
 [0.65418043 0.40111209]
 [0.80676043 0.50619972]
 [0.87652205 0.59087887]
 [0.91253159 0.66592686]
 [0.93728655 0.69343138]
 [0.9456598  0.71238938]
 [0.95751454 0.7280105 ]
 [0.9642271  0.73157436]
 [0.96952125 0.73632096]
 [0.9732351  0.723793  ]
 [0.9765312  0.73914946]
 [0.97764623 0.7329196 ]
 [0.98179712 0.72765644]

## Feature extraction: inspect the results on whole dataset

[Back to contents](#Contents)

In [13]:
data_dirname = '../data/'
model_dirname = '../models/matrix_decomposition/'
model_filename_prefix = 'model_md_snn_full_dataset'

filename_dataset = 'dataset.npz'
maxitnum = 1000
rank = 25

df = np.load(data_dirname+filename_dataset)
X, y = df['data'], df['label']
X = np.reshape(X, [X.shape[0], -1], order='F')
Nfeatures = X.shape[1]

clf = MatrixClassifierLCMS(Nfeatures, rank, maxitnum=maxitnum)
tic = time.clock()
clf.fit(X, y, verbose=0)
toc = time.clock()
tms = toc-tic
clf.saveParameters(model_dirname+'rank='+str(rank)+'_'+model_filename_prefix)
print 'Evaluation time: %.2f s' % (tms)

Evaluation time: 685.31 s


In [15]:
eps = np.spacing(1.)
def func(i, A, B, Omega):
    ii = int(i)
    if (ii >= A.size) or (ii in Omega):
        return +np.infty
    rv = np.mean(np.abs(B[:, ii])) - np.log(eps+np.abs(A[ii]))
    if np.isnan(rv):
        return +np.infty
    return rv

data_dirname = '../data/'
label_plant_filename = 'species.csv'
label_plant_dict = pd.read_csv(data_dirname+label_plant_filename, index_col=0)
label_plant_dict = label_plant_dict.to_dict()['Species name']

model_dirname = '../models/matrix_decomposition/'
model_filename_base = 'model_md_snn_full_dataset'
results_dirname = '../results/'

filename_dataset = 'dataset.npz'

rank = 25
n_comp = 3

df = np.load(data_dirname + filename_dataset)
X, y = df['data'], df['label']
X = np.reshape(X, [X.shape[0], -1], order='F')
Nfeatures = X.shape[1]

clf = MatrixClassifierLCMS(Nfeatures, rank)
model_filename = 'rank='+str(rank)+'_'+model_filename_base+'.npz'
print model_filename
clf.loadParameters(model_dirname+model_filename)

components = clf.FeatureSpaces
corrDict = {}
#Z = X[test_indices[ind]].copy()
# the first loop is for component spaces
Omegas = {}
for i_cl in xrange(len(clf.classes)):
    tmp = []
    current_component_class = clf.classes[i_cl]
    # the second loop is for samples' classes
    for j_cl in xrange(len(clf.classes)):
        current_sample_class = clf.classes[j_cl]
        which = np.where(y == current_sample_class)[0]
        corrMat = np.corrcoef(X[which].T, components[current_component_class], rowvar=False)
        corrMat = corrMat[:len(which), len(which):]
        tmp.append( np.median(corrMat, axis=0) )
    corrDict[current_component_class] = np.array(tmp)
    spind = range(i_cl) + range(i_cl+1, len(clf.classes))
    spind = tuple(spind)
    Omega = []
    f = lambda i: func(
        i,
        corrDict[current_component_class][i_cl, :],
        corrDict[current_component_class][spind, :],
        Omega
    )
    ranges = (slice(0, components[current_component_class].shape[1], 1),)
    for l_c in xrange(n_comp):
        p = scipy.optimize.brute(f, ranges)
        p = int(p)
        '''
        print (
            current_component_class,
            corrDict[current_component_class][i_cl, p],
            corrDict[current_component_class][:, p]
        )
        '''
        Omega.append(p)
    
    Omegas[current_component_class] = Omega

np.savez_compressed(
    model_dirname+'snmf_components', classes=clf.classes, components=components, Omegas=Omegas, 
    corrDict=corrDict
)


rank=25_model_md_snn_full_dataset.npz


/home/hariyuki/apd/lib/python2.7/site-packages/numpy/lib/function_base.py:2530: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/hariyuki/apd/lib/python2.7/site-packages/numpy/lib/function_base.py:2531: RuntimeWarning: divide by zero encountered in true_divide
  c /= stddev[None, :]
/home/hariyuki/apd/lib/python2.7/site-packages/numpy/lib/function_base.py:2531: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/home/hariyuki/apd/lib/python2.7/site-packages/numpy/lib/function_base.py:3405: RuntimeWarning: Invalid value encountered in median for 1 results
  r = func(a, **kwargs)
